In [4]:
import openai, langchain, pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from tqdm.auto import tqdm
from uuid import uuid4
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
from langchain.chains import RetrievalQA
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import tiktoken
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool
from langchain.agents import Tool
from langchain import OpenAI, SQLDatabase
import googleapiclient.discovery
import re
from langchain.chains import SQLDatabaseSequentialChain
from langchain.text_splitter import TokenTextSplitter
import random
import pinecone

In [3]:
YOUR_API_KEY = "" 
YOUR_ENV= "us-west1-gcp-free"

index_name = 'chatfast'

pinecone.init(
                api_key=YOUR_API_KEY,
                environment=YOUR_ENV
)

if index_name not in pinecone.list_indexes():
                # we create a new index
    pinecone.create_index(
                    name=index_name,
                    metric='dotproduct',
                    dimension=1536  # 1536 dim of text-embedding-ada-002
                )
       

In [6]:
     ##we need to define a url var
url = "https://www.youtube.com/watch?v=fuhE6PYnRMc&ab_channel=MrBeast"
print(url)
match = re.search(r"watch\?v=(.{11})", url)
result_id = match.group(1)
            ### This will be the start of if logic


https://www.youtube.com/watch?v=fuhE6PYnRMc&ab_channel=MrBeast


In [7]:
def load_file(text,source_title):     
    text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(text)
    messages=limit_check(texts)
    print(len(messages))
    api_key = "OpenAI API KEY HERE"
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    metadata = [{"source_title": source_title }] * len(messages)
    Pinecone.from_texts([t for t in messages],embeddings,index_name="chatfast",metadatas=metadata, namespace='')

def create_chunks(text):
    char_text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap=0)
    texts = char_text_splitter.split_text(text)
    return texts

def limit_check(text):
    messages=[]
    for item in text: 
        messages.append(item)  
        token_count = get_token_count(messages)
        if token_count >= 5800:
            messages.pop()
            break
        # print(token_count)
        # print(len(messages))
    return messages

def get_token_count(messages):
    tokens = num_tokens_from_messages(messages)
    return tokens


def num_tokens_from_messages(messages, model="gpt-3.5-turbo"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":  # note: future models may deviate from this
        num_tokens = 0
        for value in messages:
            num_tokens += len(encoding.encode(value))
        return num_tokens
        
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}""")


In [14]:
index = pinecone.Index('chatfast')

pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
        )
index_name = 'chatfast'
transcript = YouTubeTranscriptApi.get_transcript(result_id)
response = [d['text'] for d in transcript if 'text' in d]
text="".join(response)
source_title=result_id
texts=load_file(text,source_title)               


3


In [9]:

OPENAI_API_KEY="OpenAI API Key Here"
            # chat completion llm
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
doc_search = Pinecone.from_existing_index(index_name="chatfast", embedding=embeddings)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
            )
            # conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
            )
            # retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=doc_search.as_retriever(),
    verbose=True
            )


In [15]:
qa.run("Did anyone win money and how much?")



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, someone won money. They won $95,000.'

{}